In [7]:
import tensorflow as tf
assert tf.__version__=='2.2.0-rc1'

In [ ]:
pdf = pd.DataFrame(columns=['comi', 'comj', 'S_sum', 'S_rad_mean', 'S_hot', 'C_sum', 'S_rad_mean', 'C_hot'])
fig, ax = plt.subplots(ncols=2, nrows=5, figsize=(10, 30))
i = 0
for parsed_record in dataset.take(1):
    plt.imshow(parsed_record['image'][:,:,0])
    plt.colorbar()
#     print(parsed_record['labels'])
#     output = cluster_maker(parsed_record)
#     A = output['proto'].numpy()
#     A1 = A[A[...,3] == 0][...,:3]
#     A2 = A[A[...,3] == 1][...,:3]
#     # A1 = np.unique(A1, axis=0)
#     # A2 = np.unique(A2, axis=0)
#     # print(A1)
#     sp = tf.SparseTensor(A1[...,1:3], A1[...,0], dense_shape=[HEIGHT,WIDTH])
#     A1_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
#     sp = tf.SparseTensor(A2[...,1:3], A2[...,0], dense_shape=[HEIGHT,WIDTH])
#     A2_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
#     ax[i][0].imshow(A1_dense)
#     ax[i][1].imshow(A2_dense)
    
#     output2 = cluster_splitter(output)
# #     A = output['cluster'].numpy()
# #     A1 = A[A[...,3] == 0][...,:3]
# #     A2 = A[A[...,3] == 1][...,:3]
# # #     # A1 = np.unique(A1, axis=0)
# # #     # A2 = np.unique(A2, axis=0)

#     output3 = scalar_features(output2)
#     print(parsed_record['scalar'])
# #     u = np.unique(A1[:,0])
# #     print(u)
    
# #     sp = tf.SparseTensor(A1[...,1:3], A1[...,0], dense_shape=[HEIGHT,WIDTH])
# #     A1_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
# #     ax[i][1].imshow(A1_dense)
    
#     i += 1
#     print("eventId", parsed_record['eventId'])

## Toy example

In [2]:
HEIGHT = 20
WIDTH = 20
TENERGY = 10e3
TLOCMAX = 100
TNUMBER = 3
# good values to test if recursive function works
TSEED = tf.constant(1000, tf.float32)  # tf.constant([300.], tf.float32)
TNEIGHBOR = tf.constant(100., tf.float32)
TCELL = tf.constant(50., tf.float32)

NameError: name 'tf' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
X = np.vstack((np.random.multivariate_normal(mean=[2+2,3+2], cov=[[1,0],[0,1]], size=10000),
               np.random.multivariate_normal(mean=[2+2,7+2], cov=[[0.7,0],[0,1]], size=10000),
               np.random.multivariate_normal(mean=[8+2,4+2], cov=[[1,0],[0,1]], size=10000),
               np.random.multivariate_normal(mean=[15,15], cov=[[1,0],[0,1]], size=15000)))
A = plt.hist2d(X[:,0], X[:,1], bins=HEIGHT, range=[[0, HEIGHT], [0, WIDTH]])[0]
# plt.imshow(A)
# B = A[..., np.newaxis]
A[:2,6] = 1.5*TNEIGHBOR
A[0,7:9] = 1.5*TCELL
A[1,5] = 0
A[3,16] = 20*TSEED

B = np.stack((A,0.5*A), axis=-1).astype(np.float32)
np.shape(B)

In [3]:
plt.imshow(A > TSEED)
plt.show()
plt.imshow(A > TNEIGHBOR)
plt.show()
plt.imshow(A > TCELL)
plt.show()
plt.imshow((A > TCELL) & (A <= TNEIGHBOR))
plt.show()

NameError: name 'plt' is not defined

In [4]:
output = cluster_maker({'image': tf.constant(B)})

NameError: name 'cluster_maker' is not defined

In [5]:
A = output['proto'].numpy()
# print(A)
# A = A[...,1:]
A1 = A[A[...,3] == 0][...,:3]
A2 = A[A[...,3] == 1][...,:3]
# A1 = np.unique(A1, axis=0)
# A2 = np.unique(A2, axis=0)
# print(A1)

sp = tf.SparseTensor(A1[...,1:3], A1[...,0], dense_shape=[HEIGHT,WIDTH])
A1_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
plt.imshow(A1_dense)

NameError: name 'output' is not defined

In [6]:
sp = tf.SparseTensor(A2[...,1:3], A2[...,0], dense_shape=[HEIGHT,WIDTH])
A2_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
plt.imshow(A2_dense)

NameError: name 'tf' is not defined

In [ ]:
# output2['cluster']

In [ ]:
# output = cluster_maker({'image': B})
output2 = cluster_splitter(output)
# output2

In [ ]:
scalar_features(output2)

In [ ]:
C = output2['cluster'].numpy()
# plt.
# print(A)
# A = A[...,1:]
C1 = C[C[...,3] == 0][...,:3]
C2 = C[C[...,3] == 1][...,:3]
C1 = C1[C1[...,0] == 1]
# C1 = np.unique(C1, axis=0)
# C2 = np.unique(C2, axis=0)
# print(A1)

sp = tf.SparseTensor(C1[...,1:3], C1[...,0], dense_shape=[HEIGHT,WIDTH])
C1_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
plt.imshow(C1_dense)

In [ ]:
C = output2['cluster'].numpy()
# print(A)
# A = A[...,1:]
C1 = C[C[...,3] == 0][...,:3]
C2 = C[C[...,3] == 1][...,:3]
# A1 = np.unique(A1, axis=0)
# A2 = np.unique(A2, axis=0)
# print(A1)

sp = tf.SparseTensor(C1[...,1:3], C1[...,0], dense_shape=[HEIGHT,WIDTH])
C1_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
plt.imshow(C1_dense)

In [ ]:
sp = tf.SparseTensor(C2[...,1:3], C2[...,0], dense_shape=[HEIGHT,WIDTH])
C2_dense = tf.sparse.to_dense(tf.sparse.reorder(sp))
plt.imshow(C2_dense)

## Unit testing

In [ ]:
def test_sorting():
    # arrange
    image = tf.constant([[[3., 2.], [6., 1.]], [[5., 8.], [7., 4.]]])
    TSEED = 4.
    seed_list = tf.where(tf.math.greater(image, TSEED))
    seed_len = 4
    
    # act
    sorted_seedlist = sorting(seed_list, seed_len, image)
    
    # assert
    assert tf.equal(sorted_seedlist, tf.constant([[1,0,1], [1,1,0], [0,1,0], [1,0,0]], tf.int64)).numpy().all()

In [ ]:
def test_neighbors():
    # arrange
    index = tf.constant([0,0,0], tf.int64)
    seed_list = tf.constant([[1,0,0], [1,1,0], [1,2,0]], tf.int64)
    
    # act
    neigh = neighbors(index, seed_list)
    
    # assert
    assert tf.equal(neigh, tf.constant([[0,0,1], [0,1,0]], tf.int64)).numpy().all()

In [ ]:
test_neighbors()
# test_sorting()